# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"


In [36]:
roi = ee.FeatureCollection(f"{data_folder}/raw/biomes_br") \
    .filter(ee.Filter.eq("CD_Bioma", 6)).geometry()

In [31]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")#.clip(roi)

task = ee.batch.Export.image.toDrive(
    image = mature_biomass,
    description = 'mature_biomass_500m_todrive',
    fileNamePrefix = 'mature_biomass_500m',
    region = mature_biomass.geometry(),
    scale = 500,
    crs="EPSG:4326",
    maxPixels = 1e13
)
task.start()

# Create the export task
task = ee.batch.Export.image.toAsset(
    image=mature_biomass,
    description="mature_biomass_500m",
    assetId=f"projects/amazon-forest-regrowth/assets/mature_biomass_500m",
    scale=500,
    crs="EPSG:4326",
    maxPixels = 1e13
)
# Start the export task
task.start()

In [38]:
age_agbd = ee.Image(f"{data_folder}/age_agbd")

mature_biomass_aggregated = ee.Image(f"{data_folder}/mature_biomass_100m")

distance = mature_biomass_aggregated.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("distance") \
    .updateMask(age_agbd.select("age")).float()#.clip(roi)

tst = ee.Image(f"{data_folder}/distance_500m_amaz")
map = geemap.Map()
map.addLayer(tst, {"min": 0, "max": 10000}, "distance")
map.addLayer(mature_biomass_aggregated, {"min": 0, "max": 300}, "mature_biomass_aggregated")
map

# # Create the export task
# task = ee.batch.Export.image.toAsset(
#     image=distance,
#     description="distance_500m_toasset",
#     assetId=f"projects/amazon-forest-regrowth/assets/distance_1000m",
#     scale=1000,
#     crs="EPSG:4326",
#     maxPixels = 1e13
# )
# # Start the export task
# task.start()

# task = ee.batch.Export.image.toDrive(
#     image = distance,
#     description = 'distance_1000m_todrive',
#     fileNamePrefix = 'distance_1000m',
#     region = distance.geometry(),
#     scale = 1000,
#     crs="EPSG:4326",
#     maxPixels = 1e13
# )
# task.start()

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [27]:
def export_csv(year):
    if (year == 0):
        land_use_years = ee.Image(f"{data_folder}/land_use")
        name = "all_LULC_1000_countrywide"
    else:
        land_use_years = ee.Image(f"{data_folder}/land_use_{year}_years")
        name = f"{year}y_LULC_1000_countrywide"

    age_agbd = ee.Image(f"{data_folder}/age_agbd")
    categorical = ee.Image(f"{data_folder}/categorical")
    sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")
    one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask").rename("one_hectare_mask")
    cwd = ee.Image(f"{data_folder}/raw/cwd_chave").float().rename("cwd")
    yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
    yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
    mat = ee.Image(f"{data_folder}/nearest_mat_biomass_1000_countrywide").rename("distance", "mature_biomass")

    combined_mask = one_hectare_mask.And(land_use_years.select("last_LU")) \
        .And(mat.select("distance"))

    unified_data = (
        age_agbd.addBands(
            [mat, distance, categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
        ]).updateMask(combined_mask)
    )


    unified_data_fc = unified_data.stratifiedSample(
        numPoints = 5000, classBand = 'ecoreg', region = unified_data.geometry()
    )


    # Export an ee.FeatureCollection to Google Drive, adjusting the description based on the years
    task = ee.batch.Export.table.toDrive(
        collection=unified_data_fc, description=name
    )
    task.start()

export_csv(0)
export_csv(5)
export_csv(10)
export_csv(15)